In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import dill

# Load Feathers


columns = ['title', 'brand', 'category', 'locality', 'region', 'date_added', 'posted_date']

In [2]:
# files = [file for file in os.listdir('raw_cache') if file.endswith('feather')]
files = ['raw_cache/data_{}.feather'.format(i) for i in range(10)]
dfs = []
for file in tqdm(files, desc='load files'):
    temp = pd.read_feather(file, columns=['title', 'region', 'posted_date'])
    dfs.append(temp.dropna(subset=['posted_date']))
    df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)
df['posted_date'] = pd.to_datetime(df['posted_date'], yearfirst=True, errors='coerce')
df['day_of_week'] = df['posted_date'].dt.day_name()
df.describe()

/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


,title,region,posted_date,day_of_week
count,24200832,24200832,24200829,24200829
unique,253029,51,1010,7
top,Sales Associate,CA,2018-04-08 00:00:00,Tuesday
freq,347224,3506445,600476,4631974
first,NaN,NaN,2009-10-01 00:00:00,NaN
last,NaN,NaN,2018-12-07 00:00:00,NaN


# Visualize job posts

## Total number of job posts each day

In [4]:
per_day = pd.read_feather('grouped/posts_per_day.feather')

In [ ]:
start = pd.datetime(2017, 12, 1)
end = pd.datetime(2018, 7, 21)

x = per_day['posted_date']
y = per_day['posts']

tools = 'pan,box_zoom,wheel_zoom,hover,save,reset'

f = figure(plot_width=900, plot_height=500, x_axis_type='datetime', x_axis_label='Date',
           y_axis_label='Number of posts', tools=tools, x_range=Range1d(bounds=(0, 1)))
b = f.line(x, y)

show(f)

## Weekly distribution

In [8]:
week = pd.read_feather('grouped/weekly_distribution.feather')

/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [9]:
week['ratio'] = week['posts']/total
x = week['day_of_week']
y = week['ratio']

tools = 'hover,save,reset'

f = figure(plot_width=900, plot_height=500, x_range=x, tools=tools, x_axis_label='Day of week',
           y_axis_label='Fraction of posts')
b = f.vbar(x=x, top=y, width=0.9)

show(f)

NameError: name 'figure' is not defined